In [38]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [39]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_left.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        self.do_simulation(action, self.frame_skip)

        observation = self._get_obs()
        reward = 0
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [40]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.046 ,  0.1435, -0.1245,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [41]:
leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [55]:
for i in range(-100, 100):
    for j in range(-100, 100):
        for k in range(-100, 100):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,6):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 5:
                    env_obs = obs
            

            leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [56]:
print(len(leg_dataset))

8141912


In [57]:
#Convert left leg list to numpy array
leg_dataset_as_np = np.array(leg_dataset)

In [58]:
print(leg_dataset_as_np[0:4])

observations = (leg_dataset_as_np[:, 0:3][:])
print(observations)
actions = (leg_dataset_as_np[:, 3:][:])
print(actions)

[[ 0.13712376 -0.15670043 -0.13300666 -1.         -1.         -1.        ]
 [ 0.14307411 -0.14464797 -0.13652958 -1.         -1.         -0.9       ]
 [ 0.14786721 -0.1321218  -0.13926268 -1.         -1.         -0.8       ]
 [ 0.15149051 -0.11908522 -0.14132876 -1.         -1.         -0.7       ]]
[[ 0.13712376 -0.15670043 -0.13300666]
 [ 0.14307411 -0.14464797 -0.13652958]
 [ 0.14786721 -0.1321218  -0.13926268]
 ...
 [ 0.03177611  0.04240239  0.06983805]
 [ 0.03125204  0.04117479  0.06953189]
 [ 0.03071683  0.03995365  0.06921923]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.9 ]
 [-1.   -1.   -0.8 ]
 ...
 [ 0.99  0.99  0.97]
 [ 0.99  0.99  0.98]
 [ 0.99  0.99  0.99]]


In [59]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='tanh')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_obs, train_actions, epochs=10, batch_size=1024)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)

Epoch 1/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0248 - accuracy: 0.9077
Epoch 2/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0162 - accuracy: 0.9331
Epoch 3/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0157 - accuracy: 0.9354
Epoch 4/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0153 - accuracy: 0.9367
Epoch 5/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0149 - accuracy: 0.9380
Epoch 6/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0146 - accuracy: 0.9395
Epoch 7/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0142 - accuracy: 0.9412
Epoch 8/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0140 - accuracy: 0.9421
Epoch 9/10
7156/7156 [==============================] - 8s 1ms/step - loss: 0.0139 - accuracy: 0.9429
Epoch 10/10
25444/25444 [==============================] - 8s 295us/step - loss: 0

In [61]:
model.save("left_legs_model_plus50", overwrite=True)


converter = tf.lite.TFLiteConverter.from_saved_model("/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/left_legs_model_plus50")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
left_leg_tflite_model = converter.convert()
with open('/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/left_legs_model_plus50.tflite', 'wb') as f:
    f.write(left_leg_tflite_model)


INFO:tensorflow:Assets written to: left_legs_model_plus50/assets


INFO:tensorflow:Assets written to: left_legs_model_plus50/assets
2024-06-08 20:49:52.517104: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-08 20:49:52.517117: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 8, Total Ops 19, % non-converted = 42.11 %
 * 8 ARITH ops

- arith.constant:    8 occurrences  (f32: 8)



  (f32: 5)
  (uq_8: 2)
  (f32: 1)
